# Chapter 13 — Tracking-Based Promotion Decision

This notebook demonstrates how to use MLflow Tracking data to make
automated promotion decisions. We query the experiment for the
best-performing run, compare it against thresholds, and promote
only if it qualifies.

**Continues from:** `03_evaluate_agent` (evaluation runs logged with metrics)

In [0]:
%pip install mlflow
%restart_python

## 1. Set Up Variables

In [0]:
CATALOG = "demo"
SCHEMA  = "finance"
try:
    username = spark.conf.get("spark.databricks.notebook.userName")
except:
    username = "unknown"
if username == "unknown":
    username = spark.sql("SELECT current_user()").collect()[0][0]
print(f"Current user: {username}")

## 2. Query Experiment Runs and Promote the Winner

This turns promotion from a manual judgment call into an automated gate
in the CI/CD pipeline. We filter for runs that meet quality thresholds,
rank by latency, and promote the best one.

In [0]:
import mlflow
from mlflow import MlflowClient

client = MlflowClient()
experiment = mlflow.get_experiment_by_name(
    f"/Users/{username}/data_quality_agent_experiment"
)

# Find the run with the lowest latency that produced enough rules
best_runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string="metrics.rules_generated >= 5",  # <1>
    order_by=["metrics.latency_seconds ASC"],
    max_results=1
)

best = best_runs[0]
print(f"Best run: {best.info.run_id}")
print(f"  Latency: {best.data.metrics['latency_seconds']:.2f}s")
print(f"  Rules:   {int(best.data.metrics['rules_generated'])}")

## 3. Register and Promote the Winning Run

The alias update is what causes the production serving endpoint to pick
up the new version — no redeployment needed.

In [0]:
model_name = f"{CATALOG}.{SCHEMA}.data_quality_agent"

# Since the experimental runs don't have logged models, we promote the latest
# registered version if the best experimental run meets our criteria
if best.data.metrics['latency_seconds'] < 10 and best.data.metrics['rules_generated'] >= 5:
    # Get the latest version
    versions = client.search_model_versions(f"name='{model_name}'")
    latest_version = max([int(v.version) for v in versions])
    
    client.set_registered_model_alias(model_name, "champion", latest_version) # <2>
    print(f"Promoted version {latest_version} to 'champion' based on experimental criteria")
    print(f"  Best run {best.info.run_id} met thresholds:")
    print(f"    Latency: {best.data.metrics['latency_seconds']:.2f}s < 10s")
    print(f"    Rules: {int(best.data.metrics['rules_generated'])} >= 5")
else:
    print("Best run did not meet promotion criteria")
    print(f"  Latency: {best.data.metrics['latency_seconds']:.2f}s")
    print(f"  Rules: {int(best.data.metrics['rules_generated'])}")